In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

In [2]:
import pandas as pd
import re
import pprint

## Connexion a spotify

In [3]:
CRED_PATH_SPOTIFY = "../credentials-spotify.json"

In [4]:
with open(CRED_PATH_SPOTIFY, 'r') as handle:
    data = json.load(handle)

In [5]:
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    **data
))

## Connexion a google

In [6]:
CREDENTIALS_PATH_GOOGLE = "../credentials-sheets.json"

In [16]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET = '1b75J-QTGrujSgF9r0_JPOKkcXAwzFVwpETOAyVBw8ak'

In [17]:
# Load service account credentials.
__credentials = Credentials.from_service_account_file(CREDENTIALS_PATH_GOOGLE, scopes=SCOPES)

# Creates Google Sheets API (v4/latest) service.
service = build('sheets', 'v4', credentials=__credentials)

In [18]:
values = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET, range='Notations').execute()['values']
headers = values.pop(0)

In [19]:
df = pd.DataFrame(values, columns=headers)

# DF preprocessing

In [20]:
df.set_index(['genre', 'sub_genre', 'artist', 'album', 'song'], inplace=True)
df = df.apply(lambda s: s.str.replace(",", "."))
df = df.apply(lambda s: pd.to_numeric(s, errors='coerce'))

In [21]:
df.describe()

,Qu,Gr,Vi,Ro,Sa,Gl,Ro,Cl,Lu
count,1494.000000,737.000000,738.000000,303.000000,251.000000,113.000000,19.000000,20.000000,14.000000
mean,7.656961,8.080529,7.389566,7.758911,6.653586,7.484513,7.973684,6.435000,6.944286
std,1.356360,0.873926,1.343169,1.563735,1.945438,1.724949,2.288095,1.805044,3.057809
min,0.000000,1.000000,0.000000,0.100000,0.050000,0.000000,2.000000,0.000000,0.000000
25%,7.250000,7.750000,7.000000,7.300000,5.000000,7.000000,7.500000,5.950000,6.847500
50%,8.000000,8.250000,7.500000,8.100000,7.000000,7.500000,8.000000,6.625000,8.000000
75%,8.500000,8.750000,8.500000,8.700000,8.000000,8.500000,10.000000,7.375000,8.875000
max,10.000000,9.500000,9.750000,9.700000,10.000000,10.000000,10.000000,8.750000,9.330000


In [24]:
search = "artist:\"Black Sabbath\" track:\"Evil Woman\" album:\"Black Sabbath\""
search = "artist:\"Creedence Clearwater Revival\" album:\"Cosmos Factory\" track:\"I Heard It Through The Grapevine\""
# search = "artist:\"Creedence Clearwater Revival\" album:\"Co\" track:\"I Heard It Through The Grapevine\""
# search = "artist:\"Creedence Clearwater Revival\" track:\"I Heard It Through The Grapevine\""
# search = "artist:\"Black\%20Sabbath\"%20track:\"Evil%20Woman\"%20album:\"Black%20Sabbath\""
pprint.pp(spotify.search(search, market="US"))
# pprint.pp(spotify.search(search, market="FR"))

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3A%22Creedence+Clearwater+Revival%22+album%3A%22Cosmos+Factory%22+track%3A%22I+Heard+It+Through+The+Grapevine%22&type=track&market=US&offset=0&limit=10',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3IYUhFvPQItj6xySrBmZkd'},
                                              'href': 'https://api.spotify.com/v1/artists/3IYUhFvPQItj6xySrBmZkd',
                                              'id': '3IYUhFvPQItj6xySrBmZkd',
                                              'name': 'Creedence Clearwater '
                                                      'Revival',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:3IYUhFvPQItj6xySrBmZkd'}],
                                 'external_urls': {'spotify': 'https://open.spotify.com/albu

In [62]:
searches = ['artist:"The Who" album:"Who\'s Next" track:"Baba O Riley"',
  'artist:"The Who" album:"Whos Next" track:"Baba O Riley"']
i = 0
search = searches[i]
pprint.pp(spotify.search(search, market="FR"))

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3A%22The+Who%22+album%3A%22Who%27s+Next%22+track%3A%22Baba+O+Riley%22&type=track&market=FR&offset=0&limit=10',
            'items': [],
            'limit': 10,
            'next': None,
            'offset': 0,
            'previous': None,
            'total': 0}}


In [26]:
markets = ["FR", "US"]
def search_string(row):
    search = ""
    # artists
    artists = list(map(str.strip, row.artist.split(",")))
    if len(artists) > 1:
        sep = '" AND "'
        search += f"artist:\"{sep.join(artists)}\""
    else:
        search += f"artist:\"{artists[0]}\""
    # album
    if row.album != "N/A":
        search += f" album:\"{row.album}\""
    # track name
    search += f" track:\"{row.song}\""
    # dealing with "'""
    # sometimes it will work with the "'" and sometimes not
    if "'" in search:
        searches_s = [search, search.replace("'", "")]
    else:
        searches_s = [search]
    searches = []
    for market in markets:
        for search in searches_s:
            searches.append((search, market))
    return searches

In [28]:
bad_formats = []
for idx, (_, content) in enumerate(df.index.to_frame().iterrows()):
    # search = f"artist:\"{content.artist}\" track:\"{content.song}\" album:\"{content.album}\""
    searches = search_string(content)
    bad_format = []
    for search, market in searches:
        try:
            res = spotify.search(search, market=market)
            track = res['tracks']['items'][0]
        except IndexError as e:
            print(f"{search} not in spotify")
            bad_format.append((search, market))
        else:
            break
    else:
        bad_formats.append(bad_format)
        continue
    album = track['album']['name']
    name = track['name']
    artist = track['artists'][0]['name']
    id = track['id']
    print(f"{idx:<4}/{len(df)} : {id} {name} {artist} {album}")

0   /1698 : 0x5BKBAcia1th7UzcoHnD1 I Do, I Do, I Do, I Do, I Do ABBA Abba
1   /1698 : 2TxCwUlqaOH3TIyJqGgR91 Mamma Mia ABBA Abba
2   /1698 : 5pMmWfuL0FTGshYt7HVJ8P SOS ABBA ABBA Gold
3   /1698 : 798cuJeotvXP8UVa8GJPnD Knowing Me, Knowing You ABBA Arrival


4   /1698 : 4euAGZTszWPrriggYK0HG9 Lay All Your Love On Me ABBA Super Trouper
5   /1698 : 0J2p4KYdr6Mg4ET6JPlbe1 Super Trouper ABBA Super Trouper
6   /1698 : 3oEkrIfXfSh9zGnE7eBzSV The Winner Takes It All ABBA Super Trouper
7   /1698 : 5BckPAYcKEJuYs1eV1BHHe Take A Chance On Me ABBA The Album


8   /1698 : 3Dy4REq8O09IlgiwuHQ3sk Waterloo ABBA Waterloo
9   /1698 : 7LRMbd3LEoV5wZJvXT1Lwb T.N.T. AC/DC High Voltage
10  /1698 : 2zYzyRzz6pRmhPzyfMEC8s Highway to Hell AC/DC Highway to Hell
11  /1698 : 5jCp5VtcpUlHtW8Dwlx13Y Angry Chair Alice In Chains Dirt


12  /1698 : 22Ntyke0ZDZy2Uuf8BEDkU Dam That River Alice In Chains Dirt
13  /1698 : 2QjkH9q5Mypj6m38u7Ni9o Dirt Alice In Chains Dirt
14  /1698 : 7FRfYOql61DGDp9VPPe2qA Down In A Hole Alice In Chains Dirt


15  /1698 : 1klrDyGRsCBPofZdqdaZp6 God Smack Alice In Chains Dirt
16  /1698 : 0rglK8l5QBSADvao8n4d2N Hate To Feel Alice In Chains Dirt
17  /1698 : 1mvEbRAlocvkJvqZIj3zHu Junkhead Alice In Chains Dirt


...

artist:"Depeche Mode" album:"Just Cant Get Enough" track:"Speak & Spell" not in spotify
artist:"Depeche Mode" album:"Just Can't Get Enough" track:"Speak & Spell" not in spotify
artist:"Depeche Mode" album:"Just Cant Get Enough" track:"Speak & Spell" not in spotify
1687/1698 : 0dOg1ySSI7NkpAe89Zo0b9 Born in the U.S.A. Bruce Springsteen Born In The U.S.A.


1688/1698 : 6UB9mShVLbMm0W4e6vud4C Battery Metallica Master Of Puppets (Remastered)
1689/1698 : 2MuWTIM3b0YEAskbeeFE1i Master Of Puppets Metallica Master Of Puppets (Remastered)
1690/1698 : 6FDpQJtEKdYgtyN316VxLO The Thing That Should Not Be - Remastered Metallica Master Of Puppets (Deluxe Box Set / Remastered)


1691/1698 : 1FPSkRkDlthbAceSIIWc4C Welcome Home (Sanitarium) - Remastered Metallica Master Of Puppets (Deluxe Box Set / Remastered)
1692/1698 : 6DKt7DVgBWadoaltUYlPz8 Disposable Heroes - Remastered Metallica Master Of Puppets (Deluxe Box Set / Remastered)
1693/1698 : 6XN00D9ejjEt1cW1sY44Gu Leper Messiah Metallica Master Of Puppets (Remastered)


1694/1698 : 3z9e5b0P7zoIeV6I3iyX8O Orion - Remastered Metallica Master Of Puppets (Deluxe Box Set / Remastered)
1695/1698 : 1ogUJHv5Nu9S6yC2Cv6Pvh Damage, Inc. - Remastered Metallica Master Of Puppets (Deluxe Box Set / Remastered)
1696/1698 : 7dH9Qnbb7pwDahy1Vb9ZQa Yuve Yuve Yu The HU The Gereg


1697/1698 : 5hISmTJXBXdOes4htbUhGk Wolf Totem The HU The Gereg


In [29]:
bad_formats

[[('artist:"Auracle" album:"City Slickers" track:"Bombs Away Ballet"', 'FR'),
  ('artist:"Auracle" album:"City Slickers" track:"Bombs Away Ballet"', 'US')],
 [('artist:"Auracle" album:"City Slickers" track:"City of Penetrating Light"',
   'FR'),
  ('artist:"Auracle" album:"City Slickers" track:"City of Penetrating Light"',
   'US')],
 [('artist:"Auracle" album:"City Slickers" track:"Honey"', 'FR'),
  ('artist:"Auracle" album:"City Slickers" track:"Honey"', 'US')],
 [('artist:"Auracle" album:"City Slickers" track:"Little City Slickers"',
   'FR'),
  ('artist:"Auracle" album:"City Slickers" track:"Little City Slickers"',
   'US')],
 [('artist:"Auracle" album:"City Slickers" track:"Rotary Andy\'s Raggedy"',
   'FR'),
  ('artist:"Auracle" album:"City Slickers" track:"Rotary Andys Raggedy"',
   'FR'),
  ('artist:"Auracle" album:"City Slickers" track:"Rotary Andy\'s Raggedy"',
   'US'),
  ('artist:"Auracle" album:"City Slickers" track:"Rotary Andys Raggedy"',
   'US')],
 [('artist:"Auracle" 